# P3: OpenStreetMap Data Case Study

## Project Summary

In this project, I used the data wrangling techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean OpenStreetMap data. Then I converted the dataset from XML to CSV format, imported the cleaned .csv files into database, conducted SQL queries to provide a statistical overview of the dataset. Finally, I give some additional suggestions for improving and analyzing the data.


## Map Area
Amsterdam, North Holland, The Netherlands

https://www.openstreetmap.org/relation/271110

![](http://localhost:8888/notebooks/Documents/DataScientistLearningPath/GitHubProjects/Udacity_DataAnalystNanodegree/P3_Wrangle-OpenStreetMap-Data/osm_amsterdam.png)

## Generate Sample Data

The original size of the dataset is about 3.15 GB. So use the following code to take a systematic sample of elements from my original OSM region.


## Identifying Problems in the Map

After taking a 1% (31.5 MB) sample of the 3.15 GB dataset using sample_region.py, I used three techniques to identify problems in the sample:

* Scrolling through the Unix less command output to view portions of the data in their original form.
* Analyzing the osm_street_audit.py script output to view unusual street names and bus stop names.
* Analysing the CSV files created by the osm_csv_creater.py script to view the data (in schema.md format) before and after cleaning code was applied.

## Problems Encountered in the Map

Simplified versions of code cleaning the following problems are presented below.


## Inconsistent Street Names

Correcting street names and cases.

"Overweerse polderdijk" to "Overweersepolderdijk"
"wielewaallaan" to "Wielewaallaan"

I also present bits of the code that I used to tackle with them: (the code that follows is part of osm_ audit_streets.py)

```python
osmfile = "amsterdam_netherlands.osm"
street_re = re.compile(r'\S+(\s\S+)*')
streets = defaultdict(set)

expected_streets = ["achterburgwal", "baan", "dam", "dijk", "gracht",  
                      "haven",  "hof", "hoje",  "kade", "laan", "markt", "pad", "plein", "schans",
                      "straat", "voorburgwal", "weg", "dreef"]

def is_street(elem):
    return (elem.attrib['k'] == "addr:street")


mapping = { 
                "de ": "De",
                "tt.": "Tt.",
                "passage" : "Passage",
                "mees": "Mees",
                "Overweerse polderdijk": "Overweersepolderdijk",
                "mt.": "Mt.",
                "boulevard": "Boulevard",
                "mees": "Mees",
                "wielewaallaan": "Wielewaallaan"
    }

def update_street_type(name, mapping):
    m = street_re.search(name)
    if m:
        street_type = m.group()
        if  street_type in mapping.keys():
            print 'Before: ' , name
            name = re.sub(m.group(), mapping[m.group()], name)
            print 'After: ', name
    return name

if __name__ == '__main__':
    audit_street(osmfile)

```

## Incosistent and Incorrect Amenity Types

I also checked the amenities type names. I found few erroneous entries related to amenities and city names.

"grand Cafe" to "grand_Cafe"
"doctors;pharnacy;dentist" to "medisch_centrum"

I also present bits of the code that I used to tackle with them: (the code that follows is part of osm_ audit_amenities.py)


```python
osmfile = "amsterdam_netherlands.osm"
amenity_re = re.compile(r'\S+(\s\S+)*')
amenities = defaultdict(set)


expected_amenities = ["ambulance_station", "arts_center", "atm", "bank", "bar",
                        "beauty salon", "bowling club", "bus_station", "cafe",
                        "car_rental", "casino", "cinema","club", "club_house", 
                        "college","dentist", "doctors", "fast_food",
                        "fire_station", "fuel", "hospital", "nightclub", "park", 
                        "pharmacy", "police", "post_office", "pub", 
                        "restaurant", "school", "taxi", "telephone", "taxi", 
                        "theatre", "university"]

def is_amenity(elem):
    return (elem.attrib['k'] == "amenity")

def audit_amenity(osmfile):
    street_types = defaultdict(set)
    for elem in get_element(osmfile):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_amenity(tag):
                    audit_amenities(amenities, tag.attrib['v'])
                    # call update_amenity() function:
                    update_amenity(tag.attrib['v'],amenity_mapping)
    #pprint.pprint(dict(amenities))

#audit_amenity(osmfile)

amenity_mapping = {
                    "Art": "art",
                    "grand Cafe": "grand_cafe", 
                    "Mortgage bank": "mortgage_bank",
                    "doctors;pharnacy;dentist": "medisch_centrum",
                    "first aid": "first_aid",
                    "lock_door storage": "lockers",
                    "Broker": "broker",
                    "Healthcare": "healthcare",
                    "Wellness" : "wellness"
    		      }

def update_amenity(name, mapping):
    m = amenity_re.search(name)
    if m:
        amenity_type = m.group()
        if  amenity_type in mapping.keys():
            print 'Before: ' , name
            name = re.sub(m.group(), mapping[m.group()], name)
            print 'After: ', name
    return name

if __name__ == '__main__':
    audit_amenity(osmfile)


```


# Incorrect Postal Code Format

Separate the integers from character duplos with a space. Discard postal codes that are not in the correct Netherlands format, i.e., 1000 etc.

* 1602NP to 1602 NP

```python
"""
Discard postal codes that are not in the correct Netherlands format, i.e.,
1000 AA, where 4 integers are followed by a space and then 2 letters.
 
"""
POSTCODE = re.compile(r'^[1-9][0-9]{3}\s?[a-zA-Z]{2}$$')

def clean_postcode(postcode):
    m = POSTCODE.search(postcode)
    if m:
        if " " not in postcode:
        # Add space in middle if there is none
            postcode = postcode[:4] + " " + postcode[4:]
        return postcode
         # Ignore tag if improper postal code format
    else:
        if postcode != POSTCODE:
            return None   

```

## Overview of the data

## File Size 


| FileName                  | Size (GB)     | 
| ------------------------- |:-------------:| 
|amsterdam_netherlands.osm  | 3.15          |
|nodes.csv                  | 1.02          |
|nodes_tags.csv             | 0.08          | 
|ways.csv                   | 0.11          |
|ways_nodes.csv             | 1.37          |
|ways_tags.csv              | 0.26          |

## Number Of Unique Users

```sql
sqlite > SELECT COUNT(DISTINCT(e.uid))
		FROM (SELECT uid FROM Nodes UNION ALL SELECT uid FROM Ways) e;
```

2766

## Number Of Nodes

```sql
sqlite> SELECT COUNT(*) 
        FROM nodes;s) e;
```

12216610

## Number Of Ways

```sql
sqlite> SELECT COUNT(*) 
        FROM ways;
```

1770577

## Top 10 Contributing Users

```sql
sqlite> SELECT e.user, COUNT(*) as num
        FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
        GROUP BY e.user
        ORDER BY num DESC
        LIMIT 10;
```


| UserName      | UserID        | 
| ------------- |:-------------:| 
|florisje_BAG   |2263274        |
|Plompy_BAG     |2195029        |
|sebastic_BAG   |2178717        |
|3dShapes       |1948248        |
|rethna_BAG     |1767282        |
|padvinder      |334624         |
|PeeWee32_BAG   |310363         |
|Zugführer_BAG  |299261         |
|stroet43       |237945         |
|Chiuaua_BAG    |212260         |


## First Contribution

```sql
sqlite> SELECT timestamp 
        FROM Nodes 
        UNION SELECT timestamp 
        From ways
        ORDER BY timestamp
        LIMIT 1;
```

2007-04-01T16:49:46Z

## List of Top 20 Amenities in Amsterdam

```sql
sqlite> SELECT value, COUNT(*) as num
        FROM nodes_tags 
        WHERE key='amenity' 
        GROUP BY value
        ORDER BY num DESC 
        LIMIT 20;
```


| AmenityName    | TotalNumber   | 
| -------------  |:-------------:| 
|bench           |3548           |
|restaurant      |1970           |
|recycling       |1506           |
|waste_basket    |1369           |
|post_box        |957            |
|fast_food       |834            |
|parking         |751            |
|cafe            |735            |
|bicycle_parking |734            |
|pub             |549            |
|atm             |323            |
|fuel            |292            |
|school          |273            |
|place_of_worship|219            |
|bar             |181            |
|charging_station|177            |
|pharmacy        |175            |
|vending_machine |170            |
|toilets         |154            |
|bank            |151            |


## Most Popular Cuisines

```sql
sqlite> SELECT nodes_tags.value, COUNT(*) as num
        FROM nodes_tags
            JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
            ON nodes_tags.id=i.id
        WHERE nodes_tags.key='cuisine'
        GROUP BY nodes_tags.value
        ORDER BY num DESC
        LIMIT 10;
```



| RestName      | TotalNumber   | 
| ------------- |:-------------:| 
|italian        |140            |
|chinese        |116            |
|regional       |76             |
|french         |62             |
|international  |53             |
|pizza          |49             |
|japanese       |40             |
|indonesian     |38             |
|asian          |35             |


## Most Popular Bank

```sql
sqlite> SELECT nodes_tags.value, COUNT(*) as num
        FROM nodes_tags
            JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='bank') i
            ON nodes_tags.id=i.id
        WHERE nodes_tags.key='name'
        GROUP BY nodes_tags.value
        ORDER BY num DESC
        LIMIT 5;
```



| BankName      | TotalNumber   | 
| ------------- |:-------------:| 
|Rabobank       |48             |
|ABN AMRO       |34             |
|ING            |25             |
|ABN Amro       |11             |
|ING Bank       |9              |



## Additional Ideas

As I was running several queries like the ones above, I thought it would be interesting to check if the openstreetmap dataset for Liverpool provides website info for restaurants or pubs. So I run the query below in sqlite:

## Total Bicycle Parking

```sql
SELECT COUNT(*)
   	FROM nodes_tags
   	WHERE value='bicycle_parking';
```



734

## Top 5 Religions in Asterdam

```sql
SELECT nodes_tags.value, COUNT(*) as num
           FROM nodes_tags 
            	 JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') i 
             	ON nodes_tags.id=i.id 
           WHERE nodes_tags.key='religion' 
           GROUP BY nodes_tags.value 
           ORDER BY num DESC 
           LIMIT 5;
```


| Religion      | TotalNumber   | 
| ------------- |:-------------:| 
|christian      |177            |
|muslim         |19             |
|hindu          |8              |
|jewish         |5              |
|buddhist       |4              |

## Challeneges 

It is clear that a significant effort has been undertaken by various users to contribute data to the Amsterdam OSM area. I believe it would be be really hard for OpenStreetMap project to implement some data quality rules for OSM that restrict the data to be entered. For example, they could include postal code rules (i.e. number of characters, character format (integer vs. alphabetical charactr at certain positions)) based on the known postal format of the Netherland. With cities and provinces, similar rules/restrictions could be put in place to ensure that users enter cities/provinces that actually exist. This could easily be checked against a database of existing cities and provinces for each country.

Nontheless, as the perfect solution is not likely to be achievable in all cases, it is necessary for OpenStreetMap to understand and capture the priorities of the users and to use these to target resources in a cost-effective manner. Which means some open issues still must be taken into acccount, such as the  fitness for purpose of OSM data, the in 
influence of geography and participation on the project, the ability to continue to update the information, and licensing.

It is well established fact that finding error is far harder than entering new data. So starting with a clean slate and adding data in is easier than starting from a dataset and discovering errors- especially when (like in this example) you'd be starting from the same dataset. Some other anticipated problems with implementing such changes would be rules that are too restrictive and end up blocking valid values. Additionally, such rules could discourage users from contributing data due to the added difficulty.

The hardest problem for Amsterdam's entries is that half were written in English and the other half were written in Dutch (Despite that there was some guidelines explaining how to del with the local language issue). I had to do some rudimentary translation. The remaining problems were still more or less related to the 'uncleanliness' of the data, and were mostly dealt with using regular expressions, like catching phone numbers with wrong number of digits, standardising the formats for postcode, phone numbers, emails..etc.


## Conclusion

The Amsterdam OpenStreetMap dataset is a quite large and quite messy. While it is clear that the data is not 100% clean, I believe it was sufficiently cleaned for the purposes of this project. Via SQL query, I learned a few new things about my amsterdam. The dataset is very useful, though areas for improvement exist.

The city and postcode values could be crosschecked when inputing a new address. Most countries have public APIs to retrieve addresses from postcodes, so it could be done, with the help of contributors around the world. This improvement could prevent a lot of wrong data inputs - there are many examples in the examined dataset - and it would make the process of analysing data related to cities considerably easier and more accurate. It would definitely cause a positive impact which would affect users througout the world. On the other hand, a change like this decreases the freedom of the user when inputing new addresses, since data could only be submitted if it was in accordance with the crosschecked value from another data source. These positive and negative impacts should be weighted before implementing this kind of improvement to the process.

From the process of auditing we can see the dataset is fairly well-cleaned even though there are some minor error such as wrong street names and incosistent amenity names and postcodes. Since there are thousands of contributing users, so it is inevitable to have so many human input error. My thought is: is it possible to create a monitor system to check everybody’s contribution regularly. In addition, because OpenStreetMaps is an open source project, there’re still a lot of areas map to be improved on. I personally felt there should be some unique keys e.g for buststops or museums. They are quite generic and its beyond scope of this project to address those issues. So I hope OpenStreetMaps can obtain will improve on these areas.

![](http://localhost:8888/notebooks/Documents/DataScientistLearningPath/GitHubProjects/Udacity_DataAnalystNanodegree/P3_Wrangle-OpenStreetMap-Data/windmill_amsterdam.png)